# Largest Square in a Matrix
Determine the area of the largest square of 1's in a binary matrix.

## Intuition

The brute-force solution to this problem involves checking every possible submatrix to determine whether it forms a square composed entirely of 1s.  
This can be done by treating each cell as the top-left corner of a square and expanding in both dimensions to verify whether all elements in the square are 1s.

However, this is highly inefficient — with a time complexity of **O(n³)** or worse — so we look for a more optimal solution.

---

### Key Observation

Squares are **composed of smaller squares**. This implies that we can **build larger squares from smaller ones**, which is a strong indication of the presence of **overlapping subproblems** — a classic case for **Dynamic Programming (DP)**.

Specifically, for any given cell `(i, j)`, if `matrix[i][j] == 1`, then the size of the square that ends at `(i, j)` is determined by the **minimum size** of the squares ending at:
- The cell directly **above**: `(i-1, j)`
- The cell directly to the **left**: `(i, j-1)`
- The cell at the **top-left diagonal**: `(i-1, j-1)`

This leads to the following recurrence relation:

```python
if matrix[i][j] == 1:
    max_square(i, j) = 1 + min(
        max_square(i - 1, j),
        max_square(i - 1, j - 1),
        max_square(i, j - 1)
    )
```

---

### DP Formulation
We define a 2D DP array `dp` where `dp[i][j]` represents the side length of the largest square that ends at position `(i, j)`.
The transition becomes:
```python
if matrix[i][j] == 1:
    dp[i][j] = 1 + min(
        dp[i-1][j],
        dp[i-1][j-1],
        dp[i][j-1]
    )
else:
    dp[i][j] = 0
```

We keep track of a `max_len` variable that stores the largest side length encountered during DP table computation. At the end, the area of the largest square is simply `max_len ** 2`.

---

### Base case
The first row and column are special cases:
- For any cell `(i, 0)` or `(0, j)`, if `matrix[i][j] == 1`, then `dp[i][j] = 1`. These cells can form only 1x1 squares, since they lack neighbors on the top, left, or diagonal.

---

### Filling the DP Table
1. Initialize the first row and column based on the input matrix.
2. Starting from cell `(1, 1)`, fill in the DP table row by row, applying the recurrence relation.
3. Track `max_len` during the iteration.
4. Return `max_len ** 2` at the end — the area of the largest square of 1s.

In [1]:
from typing import List

def largest_square_in_a_matrix(matrix: List[List[int]]) -> int:
    if not matrix:
        return 0

    m, n = len(matrix), len(matrix[0])
    dp = [[0] * n for _ in range(m)]
    max_len = 0

    for j in range(n):
        if matrix[0][j] == 1:
            dp[0][j] = 1
            max_len = 1
    
    for i in range(m):
        if matrix[i][0] == 1:
            dp[i][0] = 1
            max_len = 1

    for i in range(1, m):
        for j in range(1, n):
            if matrix[i][j] == 1:
                dp[i][j] = 1 + min(dp[i - 1][j], dp[i - 1][j - 1], dp[i][j - 1])
            
            max_len = max(max_len, dp[i][j])
    
    return max_len ** 2

### Complexity Analysis
The time complexity is **O(m × n)** because each cell of the DP table is populated at most once.

The space complexity is **O(m × n)** since we're maintaining a 2D DP table that has `m × n` elements.

---

### Optimization
We can optimize our solution by realizing that, for each cell in the DP table, we only need to access:
- the cell directly **above** it,
- the cell to its **left**,
- and the **top-left diagonal** cell.

To get:
- the **above** or **top-left diagonal** cell, we only need access to the **previous row**,
- the **left** cell is in the **current row** being populated.

Therefore, we only need to maintain two rows:
- `prev_row`: the previous row,
- `curr_row`: the current row being populated.

This effectively reduces the space complexity to **O(m)**.

In [2]:
from typing import List

def largest_square_in_a_matrix(matrix: List[List[int]]) -> int:
    if not matrix:
        return 0

    m, n = len(matrix), len(matrix[0])
    prev_row = [0] * n
    max_len = 0

    for i in range(m):
        curr_row = [0] * n
    

        for j in range(n):              
            if i == 0 or j == 0:
                curr_row[j] = matrix[i][j]
            else:
                if matrix[i][j] == 1:
                    curr_row[j] = 1 + min(curr_row[j - 1], prev_row[j - 1], prev_row[j])        
            max_len = max(max_len, curr_row[j])
        
        prev_row, curr_row = curr_row, [0] * n

    return max_len ** 2